In [ ]:
import pyvista as pv
import vtk
import numpy as np

from trame.app import get_server
from pyvista.trame.ui import plotter_ui
from trame.ui.vuetify3 import SinglePageWithDrawerLayout
from trame.widgets import vuetify3
from pyvista.plotting.themes import DocumentTheme # Creating a theme
from pyvista.plotting.colors import hexcolors
from pyvista.plotting.opts import ElementType # For element_picking (selecting a 'face')

# Server Trame Setup
server = get_server()
state, ctrl = server.state, server.controller

# Theme testing
my_theme = DocumentTheme()
my_theme.background = '#dddddd'
# my_theme.show_vertices = True
my_theme.show_edges = True
my_theme.split_sharp_edges = True
my_theme.edge_color = 'k'
my_theme.enable_camera_orientation_widget = True # Creates the camera bars in TOP RIGHT
pv.global_theme.load_theme(my_theme)
pv.set_jupyter_backend('client')

# Read the VTK file using PyVista
filename = "file.vtk"
mesh = pv.read(filename)

# Extract data arrays
dataset_arrays = []
point_data = mesh.point_data
cell_data = mesh.cell_data

# Extract point data
for i, (name, array) in enumerate(point_data.items()):
    array_range = np.min(array), np.max(array)
    dataset_arrays.append(
        {
            "text": name,
            "value": i,
            "range": list(array_range),
            "type": vtk.vtkDataObject.FIELD_ASSOCIATION_POINTS,
        }
    )

# Extract cell data
for i, (name, array) in enumerate(cell_data.items()):
    array_range = np.min(array), np.max(array)
    dataset_arrays.append(
        {
            "text": name,
            "value": i,
            "range": list(array_range),
            "type": vtk.vtkDataObject.FIELD_ASSOCIATION_CELLS,
        }
    )

default_array = dataset_arrays[0]
default_min, default_max = default_array.get("range")

# Create the plotter
plotter = pv.Plotter(notebook=True)

# Add the default mesh
dActor = plotter.add_mesh(mesh, scalars=default_array.get("text"), cmap="rainbow")

# Threshold for Z-Layer
threshold_value = 0.5 * (default_min + default_max)
z_layer = mesh.threshold([threshold_value, default_max], scalars='tentlevel')

# Add the Z-Layer mesh
zActor = plotter.add_mesh(z_layer, scalars=default_array.get("text"), cmap="rainbow", opacity=0.5)

# Set up the camera and view
plotter.view_xy()
plotter.add_axes()
plotter.show_grid()

# Define update functions
@state.change("mesh_representation")
def update_mesh_representation(representation):
    plotter.remove_actor(dActor)
    if representation == "Points":
        plotter.add_mesh(mesh, scalars=default_array.get("text"), cmap="rainbow", style='points')
    elif representation == "Wireframe":
        plotter.add_mesh(mesh, scalars=default_array.get("text"), cmap="rainbow", style='wireframe')
    elif representation == "Surface":
        plotter.add_mesh(mesh, scalars=default_array.get("text"), cmap="rainbow", style='surface')
    elif representation == "SurfaceWithEdges":
        plotter.add_mesh(mesh, scalars=default_array.get("text"), cmap="rainbow", style='surface')
        plotter.add_mesh(mesh.extract_edges(), color='black')

@state.change("zlayer_representation")
def update_zlayer_representation(representation):
    plotter.remove_actor(zActor)
    if representation == "Points":
        plotter.add_mesh(z_layer, scalars=default_array.get("text"), cmap="rainbow", style='points')
    elif representation == "Wireframe":
        plotter.add_mesh(z_layer, scalars=default_array.get("text"), cmap="rainbow", style='wireframe')
    elif representation == "Surface":
        plotter.add_mesh(z_layer, scalars=default_array.get("text"), cmap="rainbow", style='surface')
    elif representation == "SurfaceWithEdges":
        plotter.add_mesh(z_layer, scalars=default_array.get("text"), cmap="rainbow", style='surface')
        plotter.add_mesh(z_layer.extract_edges(), color='black')

def update_zlayer(z_value, **kwargs):
    global z_layer, zActor
    plotter.remove_actor(zActor)
    z_layer = mesh.threshold([z_value, default_max], scalars='tentlevel')
    zActor = plotter.add_mesh(z_layer, scalars=default_array.get("text"), cmap="rainbow", opacity=0.5)
    plotter.render()


def standard_buttons():
    """
    Define standard buttons for the GUI, including a checkbox for dark mode and a button to reset the camera.
    """
    vuetify3.VCheckbox(
        v_model="$vuetify.theme.dark",
        on_icon="mdi-lightbulb-off-outline",
        off_icon="mdi-lightbulb-outline",
        classes="mx-1",
        hide_details=True,
        dense=True,
    )
    with vuetify3.VBtn(icon=True, click="$refs.view.resetCamera()"):
        vuetify3.VIcon("mdi-crop-free")

def ui_card(title, ui_name):
    """
    Create a UI card component for organizing GUI elements.

    Args:
        title (str): The title of the card.
        ui_name (str): The name used to show/hide the card based on the active UI state.

    Returns:
        vuetify.VCardText: The content area of the card.
    """
    with vuetify3.VCard(v_show=f"active_ui == '{ui_name}'"):
        vuetify3.VCardTitle(
            title,
            classes="grey lighten-1 py-1 grey--text text--darken-3",
            style="user-select: none; cursor: pointer",
            hide_details=True,
            dense=True,
        )
        content = vuetify3.VCardText(classes="py-2")
    return content

def d_card():
    """
    Define the UI card for the default mesh settings, including options for representation, color, and opacity.
    """
    with ui_card(title="Default", ui_name="default"):
        with vuetify3.VRow(classes="pt-2", dense=True):
            with vuetify3.VCol(cols="6"):
                vuetify3.VSelect(
                    # Color By
                    label="Color by",
                    v_model=("mesh_color_array_idx", 0),
                    items=("array_list", dataset_arrays),
                    hide_details=True,
                    dense=True,
                    outlined=True,
                    classes="pt-1",
                )

def z_card():
    """
    Define the UI card for the Z-layer settings, including options for representation, color, opacity, and Z-layer value.
    """
    with ui_card(title="Z-Layer", ui_name="zlayer"):
        vuetify3.VSlider(
            # layers
            v_model=("z_value", 0.0),
            min=0,
            max=30,
            step=1,
            label="Z-Layer",
            classes="mt-1",
            hide_details=True,
            dense=True,
            thumb_label=True,
            reverse=True,
        )

with SinglePageWithDrawerLayout(server) as layout:
    layout.title.set_text("Viewer")

    with layout.toolbar:
        # toolbar components
        vuetify3.VSpacer()
        vuetify3.VDivider(vertical=True, classes="mx-2")
        standard_buttons()
        vuetify3.VSelect(
            label="Color",
            v_model=("color", "seagreen"),
            items=("array_list", list(hexcolors.keys())),
            hide_details=True,
            density="compact",
            outlined=True,
            classes="pt-1 ml-2",
            style="max-width: 250px",
        )


    with layout.drawer as drawer:
        # drawer components
        drawer.width = 325
        vuetify3.VDivider(classes="mb-2")
        d_card()
        z_card()

    with layout.content:
        # content components
        with vuetify3.VContainer(
            fluid=True,
            classes="pa-0 fill-height",
        ):
            # Default Server Rendering = True for Remote, False for Local
            # Can also use model='client' for full local, do not know if there is a difference
            view = plotter_ui(plotter, model='trame', default_server_rendering=False)
            ctrl.view_update = view.update
            ctrl.view_reset_camera = view.reset_camera

In [ ]:
await layout.ready
layout